REQUIRMENTS

In [ ]:
%pip install tflearn

In [ ]:
%pip uninstall tensorflow

In [ ]:
%pip install tensorflow

In [ ]:
%pip install nltk

In [ ]:
#nltk.download()

RULE BASED MODEL

In [37]:
# Importing modules
import re
from nltk.corpus import wordnet

import itertools

# Building a list of Keywords
list_words=[
    'hello','bye',
    'suicide','kill','die','want','feel',
    'stupid','dumb','asshole',
    'think', 'was','thought','thinking'
    ]
list_syn={}
for word in list_words:
    synonyms=[]
    for syn in wordnet.synsets(word):
        for lem in syn.lemmas():
            # Remove any special characters from synonym strings
            lem_name = re.sub('[^a-zA-Z0-9 \n\.]', ' ', lem.name())
            synonyms.append(lem_name)
    list_syn[word]=set(synonyms)
print (list_syn)

# Building dictionary of Intents & Keywords
keywords={}
keywords_dict={}
# Defining a new key in the keywords dictionary
keywords['greetings']=[]
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(list_syn['hello']):
    keywords['greetings'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['goodbyes']=[]
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(list_syn['bye']):
    keywords['goodbyes'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['insult']=[]
# Creating a list of synonyms with multiple related words
new_list_syn = itertools.chain(list_syn['stupid'], list_syn['dumb'], list_syn['asshole'])
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(new_list_syn):
    keywords['insult'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['suicide']=[]
# Creating a list of synonyms with multiple related words
new_list_syn = itertools.chain(list_syn['suicide'], list_syn['kill'], list_syn['die'], list_syn['want'], list_syn['feel'])
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(new_list_syn):
    keywords['suicide'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['automaticThought']=[]
# Creating a list of synonyms with multiple related words
new_list_syn = itertools.chain(list_syn['think'], list_syn['was'], list_syn['thought'], list_syn['thinking'])
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(new_list_syn):
    keywords['automaticThought'].append('.*\\b'+synonym+'\\b.*')

for intent, keys in keywords.items():
    # Joining the values in the keywords dictionary with the OR (|) operator updating them in keywords_dict dictionary
    keywords_dict[intent]=re.compile('|'.join(keys))
print (keywords_dict)

# Building a dictionary of responses
responses={
    'greetings':'Hello! So, tell me how have you been feeling?',
    'suicide':'If you are feeling vulnerable and need help, Please call 112 or 914 590 055 to seek help.',
    'fallback':'I dont quite understand. Could you repeat that?',
    'goodbyes':'Bye. Hope I\'ve been helpful.',
    'insult': 'Oh! that\'s mean :(',
    'automaticThought': 'When you were in that situtation and that thought came to you, how did you feel?'
}

def get_response(msg):
    matched_intent = None 
    for intent,pattern in keywords_dict.items():
        # Using the regular expression search function to look for keywords in user input
        if re.search(pattern, msg): 
            # if a keyword matches, select the corresponding intent from the keywords_dict dictionary
            matched_intent=intent  
    # The fallback intent is selected by default
    key='fallback' 
    if matched_intent in responses:
        # If a keyword matches, the fallback intent is replaced by the matched intent as the key for the responses dictionary
        key = matched_intent
        return responses[key]
    # The chatbot prints the response that matches the selected intent
    #print (responses[key])
    
    return 'none'


{'hello': {'hello', 'howdy', 'hullo', 'how do you do', 'hi'}, 'bye': {'adios', 'au revoir', 'auf wiedersehen', 'arrivederci', 'good by', 'so long', 'adieu', 'pass', 'bye', 'bye bye', 'cheerio', 'goodby', 'good bye', 'good day', 'sayonara', 'goodbye'}, 'suicide': {'suicide', 'felo de se', 'self annihilation', 'self destruction'}, 'kill': {'kill', 'vote down', 'drink down', 'obliterate', 'belt down', 'stamp out', 'pop', 'killing', 'shoot down', 'toss off', 'vote out', 'putting to death', 'down', 'defeat', 'bolt down', 'wipe out', 'pour down'}, 'die': {'pop off', 'go', 'give out', 'break down', 'choke', 'fail', 'die out', 'croak', 'dice', 'pass away', 'give way', 'die', 'give up the ghost', 'exit', 'buy the farm', 'expire', 'conk', 'cash in one s chips', 'pall', 'pass', 'become flat', 'perish', 'go bad', 'kick the bucket', 'drop dead', 'break', 'decease', 'snuff it', 'conk out'}, 'want': {'deficiency', 'deprivation', 'require', 'need', 'privation', 'desire', 'neediness', 'want', 'wishing'

CHAT GUI

In [40]:
from tkinter import *
from nltk.sentiment import SentimentIntensityAnalyzer

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

bot_name = "Kai"
script_answers = [ 
              "I’m sorry that you feel this way. Did anything in particular trigger this feeling?\
               \nPlease provide a detailed explanation of the situation.",
              "What were you thinking at the time?",
              "Please, Rate how intense this feeling was (1-100)",
              "What has happened to make you believe the thought is true?",
              "What evidence is there that this thought is not true?",
              "Taking the information into account, is there an alternative way of thinking about the situation? Please give an example",
              "How would you rate your mood now?",
             ]

class ChatGUI:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        self.msg_entry.delete(0, END)
        self._insert_kai_message("Hello!\
            \nMy name is Kai and I'm going to give you support to your self-directed CBT Therapy session.\
            \nSo, how have you been feeling?")
        # Steps that indicate in which stage the therapy session is
        self.step = 0
        self.sia = SentimentIntensityAnalyzer()


    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)
        
        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="Kai", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
    
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
       
    def _insert_message(self, msg, sender):
        if not msg:
            return
        
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        if (self.step == 3):
            self._insert_kai_message(self.check_rating(msg))
        else:
            kai_answer = get_response(msg)
            if (kai_answer == 'none'):
                if (self.step == 0 and self.sia.polarity_scores(msg)["compound"] > 0):
                    self._insert_kai_message("I'm glad that you feel this way :)")
                else:
                    self._insert_kai_message(script_answers[self.step])
                    self.step = self.step + 1           
            else: 
                self._insert_kai_message(kai_answer)
                    
        self.text_widget.see(END)
          
    def _insert_kai_message(self, msg):
        msg1 = f"{bot_name}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
    
    def check_rating(self, msg):
        kai_answer = "Please enter a valid answer"
        if (msg.isnumeric()):
                rating = int(msg)
                if (rating >= 1 and rating <= 100):
                    kai_answer = script_answers[self.step]
        return kai_answer


    

     
if __name__ == "__main__":
    app = ChatGUI()
    app.run()